In [321]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
%run trips_duration.ipynb

object


/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_93735/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
def merging_data(final_1plus_df):
    # Create a copy
    tourist_num_copy = final_1plus_df.copy()

    # Scale numerical columns (assumes 'Tourism St.' is the one to scale)
    scaler = StandardScaler()
    tourist_num_copy['Tourism St. Scaled'] = scaler.fit_transform(tourist_num_copy[['Tourism St.']])

    # Keep only relevant columns and sort by year
    tourist_num = tourist_num_copy[['Country', 'Year', 'Tourism St. Scaled']].sort_values(by=["Year"]).reset_index(drop=True)

    # Prepare raw, unfiltered predictions from original data
    unfiltered_predicted_tourist = tourist_num_copy[['Country', 'Year', 'NumTrips']] if 'NumTrips' in tourist_num_copy.columns else pd.DataFrame()

    return tourist_num, unfiltered_predicted_tourist



In [7]:
datas = merging_data(final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data

NameError: name 'StandardScaler' is not defined

In [ ]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['Bias'] = 1
    
    index_country = country_df.index[country_df['Year'].astype(int) == 2015]
    index = index_country[0]

    y_vector = []
    x_matrix = []
    x_list = []
    for i in range(index, len(country_df)): 
        if i - lag < 0:
            continue
        bias = 1 
        country = country_df.iloc[i]["Country"]
        lag_values = [country_df.iloc[i - j]['Tourism St.'] for j in range(1, lag + 1)]
        covid = country_df.iloc[i]['Covid']
        year = country_df.iloc[i]['Year']
        
        x_list = [bias, country, year, covid] + lag_values
        x_matrix.append(x_list)
        y_vector.append(country_df.iloc[i]["Tourism St."])

    x_matrix = pd.DataFrame(x_matrix)
    x_matrix.rename(columns={0: "Bias", 1: "Country", 2:"Year", 3:"Covid", 4:"lag1", 5:"lag2", 6:"lag3"}, inplace=True)


    return x_matrix, y_vector

In [262]:
data[data['Country']=='Slovakia']

,Country,Year,Tourism St.
294,Slovakia,2012,0.021458
295,Slovakia,2013,0.020559
296,Slovakia,2014,0.021513
297,Slovakia,2015,0.025319
298,Slovakia,2016,0.030385
299,Slovakia,2017,0.034212
300,Slovakia,2018,0.039610
301,Slovakia,2019,0.044606
302,Slovakia,2020,0.019206
303,Slovakia,2021,0.023941


In [322]:
X, y = get_lag_columns(data[data['Country'] == 'Slovakia'], lag=3)
print(X)
print(y)

   Bias   Country  Year  Covid      lag1      lag2      lag3
0     1  Slovakia  2015      0  0.021513  0.020559  0.021458
1     1  Slovakia  2016      0  0.025319  0.021513  0.020559
2     1  Slovakia  2017      0  0.030385  0.025319  0.021513
3     1  Slovakia  2018      0  0.034212  0.030385  0.025319
4     1  Slovakia  2019      0  0.039610  0.034212  0.030385
5     1  Slovakia  2020      1  0.044606  0.039610  0.034212
6     1  Slovakia  2021      1  0.019206  0.044606  0.039610
7     1  Slovakia  2022      0  0.023941  0.019206  0.044606
8     1  Slovakia  2023      0  0.041598  0.023941  0.019206
[0.02531901720004173, 0.03038532634637201, 0.03421163044933105, 0.0396101720567253, 0.04460578041262051, 0.019206153354248967, 0.023941342883577185, 0.04159770013749046, 0.04807679240371502]


ValueError: could not convert string to float: 'Slovakia'

In [264]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [265]:
def predict_next_year(beta, X):
    return np.dot(beta, X)

In [ ]:
def get_country_years_pred(data, country, lag = 3):

    country_df = data[data['Country'] == country]
    country_df, y = get_lag_columns(country_df, lag = lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias'] + lag_cols
    
    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)

    predictions = []
    for index, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row[col] for col in feature_cols])
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_Tourism': y[index],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [276]:
prev_year_result = get_country_years_pred(data, "Slovenia")
prev_year_result

,Country,Year,Actual_Tourism,Predicted_Tourism
0,Slovenia,2015,0.013468,0.016434
1,Slovenia,2016,0.014178,0.015378
2,Slovenia,2017,0.015062,0.016397
3,Slovenia,2018,0.016154,0.017674
4,Slovenia,2019,0.015126,0.018117
5,Slovenia,2020,0.009315,0.016696
6,Slovenia,2021,0.021268,0.011013
7,Slovenia,2022,0.024754,0.021216
8,Slovenia,2023,0.032872,0.029272


In [314]:
def get_country_prediction(data, country, lag=3):
    country_df = data[data['Country'] == country].copy()
    country_df, y = get_lag_columns(country_df, lag=lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias'] + lag_cols

    X = country_df[feature_cols].values
    y_tourism = np.array(y)

    beta_tourism = fit_regression(X, y_tourism)

    last_year = country_df['Year'].max()

    last_row = country_df[country_df['Year'] == last_year].iloc[0]
    lag_values = [last_row[f'lag{i}'] for i in range(1, lag + 1)]
    
    predictions = []

    for i in range(1, lag + 1):
        year = last_year + i
        
        X_new = np.array([1] + lag_values)
        pred_tourism = predict_next_year(beta_tourism, X_new)

        predictions.append({
            'Year': year,
            'Country': country,
            'Predicted_Tourism': pred_tourism
        })

        tourism_lag_val = [pred_tourism] + lag_values[:-1]

    return predictions


In [315]:
result = get_country_prediction(data, "Italy", lag=3)
result = pd.DataFrame(result)
result.sort_values(by=['Year'])
result

,Year,Country,Predicted_Tourism
0,2024,Italy,0.19824
1,2025,Italy,0.19824
2,2026,Italy,0.19824


In [316]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    
    country_name = st_data["Country"].iloc[0]
    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)
    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Tourism_combined'], 
        name='Tourist Numbers',
        line=dict(color='royalblue', width=4),
    ))

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)

    fig.show()


In [317]:
visualization(data_not_st, result)

In [318]:
def cross_validate_time_series(country_df, lag = 3):
    results = []

    country_df, y_full = get_lag_columns(country_df, lag=lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias'] + lag_cols

    for i in range(lag, len(country_df) - 1):
        train_df = country_df.iloc[:i]
        test_row = country_df.iloc[i]

        X_train = train_df[feature_cols].values
        y_train = np.array(y_full[:i])

        X_test = np.array([test_row[col] for col in feature_cols])

        beta_tourism = fit_regression(X_train, y_train)
        pred_tourism = predict_next_year(beta_tourism, X_test)

        results.append({
            'Year': test_row['Year'],
            'Actual_Tourism': y_full[i],
            'Predicted_Tourism': pred_tourism
        })

    return pd.DataFrame(results)

In [319]:
def evaluate_predictions(pred_df):
    r2_tourism = r2_score(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])
    mse_tourism = mean_squared_error(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])

    evaluations = {
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [320]:
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]

for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
    # country_df

    pred_df = cross_validate_time_series(country_df, lag= 3)
    metrics = evaluate_predictions(pred_df)

    #print(pred_df)
    print(country)
    print(metrics)

Austria
{'Tourism': {'R2': -4.475050525939516, 'MSE': 0.00240190635918188}}


Belgium
{'Tourism': {'R2': -111.53917625184963, 'MSE': 0.023610275858963522}}
Bulgaria
{'Tourism': {'R2': -91.17016584253369, 'MSE': 0.0012322693672946754}}
Croatia
{'Tourism': {'R2': -6.677834776160904, 'MSE': 5.8313756305678724e-05}}
Czechia
{'Tourism': {'R2': -22.754692566673956, 'MSE': 0.006731252601579715}}
Denmark
{'Tourism': {'R2': -77.99945702362912, 'MSE': 0.01690181427605413}}
Estonia
{'Tourism': {'R2': -220.05979295385467, 'MSE': 0.003812553161021056}}
Finland
{'Tourism': {'R2': -32.68479338998906, 'MSE': 0.005393068330627044}}
France
{'Tourism': {'R2': -19.296675479263786, 'MSE': 0.13089183110621078}}
Germany
{'Tourism': {'R2': -72.28978396551659, 'MSE': 1.2156892397722543}}
Greece
{'Tourism': {'R2': -5.223068190982716, 'MSE': 6.0384431184660246e-05}}
Hungary
{'Tourism': {'R2': -109.39221890797467, 'MSE': 0.019258899151785195}}
Ireland
{'Tourism': {'R2': -256.02319149110144, 'MSE': 0.08937471629029341}}
Italy
{'Tourism': {'R2': -2.2310107231973086, 'MSE': 0.0055186180590580

LinAlgError: Singular matrix

In [312]:
X = country_df[['Bias', 'Tourism_lag']].values
y_tourism = country_df['Tourism St.'].values
y_tourism

TypeError: tuple indices must be integers or slices, not list